# Flat Image Net - Basic Graph


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.nn_visualization import variable_summaries, img_summaries
from utils.data import init_model_logging
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

## Load data

In [ ]:
data = input_data.read_data_sets('/data/fashion/', one_hot=True)
class_id2class_name_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

## Custom layer functions

In [ ]:
def linear_layer(name, input_data, shape):
    w_name = 'w_' + name
    b_name = 'b_' + name
    
    w = tf.get_variable(w_name, initializer=tf.truncated_normal(shape, stddev=0.1))
    bias = tf.get_variable(b_name, initializer=tf.constant_initializer(0), shape=shape[1])
    
    output_data = tf.matmul(input_data, w) + bias
    return output_data

In [ ]:
def sigmoid_layer(name, input_data, shape):
    w_name = 'w_' + name
    b_name = 'b_' + name
    
    w = tf.get_variable(w_name, initializer=tf.truncated_normal(shape, stddev=0.1))
    bias = tf.get_variable(b_name, initializer=tf.constant_initializer(0), shape=shape[1])
    
    output_data = tf.matmul(input_data, w) + bias
    output_data = tf.nn.sigmoid(output_data)
    return output_data

In [ ]:
def relu_layer(name, input_data, shape):
    w_name = 'w_' + name
    b_name = 'b_' + name
    
    w = tf.get_variable(w_name, shape=shape, initializer=tf.contrib.layers.xavier_initializer())
    bias = tf.get_variable(b_name, initializer=tf.constant_initializer(0), shape=shape[1])
    
    output_data = tf.matmul(input_data, w) + bias
    output_data = tf.nn.relu(output_data)
    return output_data

## Build Net Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('flat_image_net_inputs'):
        images = tf.placeholder(tf.float32, shape=[None, 784], name='images')
        labels = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
    
    with tf.variable_scope('simple_layer_1'):
        ###############################################
        # Replace linear layer with something bettet  #
        ###############################################
        raw_prediction = linear_layer(name='layer1', input_data=images, shape=[784, 64])
    
    with tf.variable_scope('simple_layer_2'):
        raw_prediction = linear_layer(name='layer2', input_data=raw_prediction, shape=[64, 10])
            
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(raw_prediction)

    with tf.name_scope('loss'):
        cross_entropy_vector = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=prediction)
        loss = tf.reduce_mean(cross_entropy_vector)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        
    with tf.name_scope('training'):
        ####################
        # Change optimizer #
        ####################
        train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
            
    initialize_vars = tf.global_variables_initializer()

## Debug Net

In [ ]:
session = tf.Session(graph=graph)
session.run(initialize_vars)

In [ ]:
_images, _labels = data.train.next_batch(10)
feed_dict = {images: _images, labels: _labels}
print("Debug [cross_entropy_vector]:", session.run(tf.shape(cross_entropy_vector), feed_dict))

In [ ]:
session.close()

## Run Net

In [ ]:
with tf.Session(graph=graph) as session:
    session.run(initialize_vars)
    for iteration in range(5001):
        ##################
        # Training Phase #
        ##################
        _images, _labels = data.train.next_batch(100)
        _train_step = session.run([train_step], feed_dict={images: _images, labels: _labels})
        
        ####################
        # Validation Phase #
        ####################
        if iteration % 100 == 0:
            _accuracy, _loss = session.run([accuracy, loss], 
                                           feed_dict={
                                               images: data.validation.images, 
                                               labels: data.validation.labels})
            print("Valid Iteration {}: loss {}, accuracy {}".format(iteration, _loss, _accuracy))
       
    # Last model prediction
    _prediction, = session.run([prediction], feed_dict={images: data.validation.images})

In [ ]:
_prediction

In [ ]:
np.sum(_prediction, axis=1)

## Results evaluation

In [ ]:
from utils.results_evaluation import get_info_df
from utils.results_evaluation import plot_examples

### Load reuslts to pandas df

In [ ]:
df = get_info_df(data.validation.labels, _prediction, class_id2class_name_mapping, images=data.validation.images)
df

In [ ]:
df[df.label_class_name=='Dress'].label_class_score.hist()

In [ ]:
plot_examples(df.iloc[:25])

### Precision

In [ ]:
from utils.results_evaluation import get_precision

In [ ]:
get_precision(df, 'Dress')

### Recall

In [ ]:
from utils.results_evaluation import get_recall

In [ ]:
get_recall(df, 'Dress')

### Overall Recall and Precision

In [ ]:
from utils.results_evaluation import get_rec_prec

In [ ]:
get_rec_prec(df, class_id2class_name_mapping)

### Accuracy

In [ ]:
from utils.results_evaluation import get_accuracy

In [ ]:
get_accuracy(df, use_top3=False)

### False Positives

In [ ]:
from utils.results_evaluation import get_false_positives

In [ ]:
fp = get_false_positives(df, label_class_name='Shirt')

In [ ]:
plot_examples(fp)

In [ ]:
fp = get_false_positives(df, label_class_name='Shirt', predicted_class_name='Pullover')

In [ ]:
plot_examples(fp)

### Confusion Matrix

In [ ]:
from utils.results_evaluation import plot_coocurance_matrix

In [ ]:
plot_coocurance_matrix(df, use_log=False, use_top3=False)